In [ ]:
import torch
try:
    # Get GPU name, check if it's K80
    GPU_name = torch.cuda.get_device_name()
    if GPU_name[-3:] == "K80":
        print("Get K80! :'( RESTART!")
        exit()  # Restart the session
    else:
        print("Your GPU is {}!".format(GPU_name))
        print("Great! Keep going~")
except RuntimeError as e:
    if e.args == ("No CUDA GPUs are available",):
        print("You are training with CPU! "
              "Please restart!")
        exit()  # Restart the session
    else:
        print("What's wrong here?")
        print("Error message: \n", e)

Your GPU is Tesla T4!
Great! Keep going~


# **Homework 10 - Adversarial Attack**

Slides: https://reurl.cc/v5kXkk

Videos:

TA: ntu-ml-2021spring-ta@googlegroups.com

## Enviroment & Download

We make use of [pytorchcv](https://pypi.org/project/pytorchcv/) to obtain CIFAR-10 pretrained model, so we need to set up the enviroment first. We also need to download the data (200 images) which we want to attack.

In [ ]:
# set up environment
!pip install pytorchcv

# download
!gdown --id 1fHi1ko7wr80wXkXpqpqpOxuYH1mClXoX -O data.zip

# unzip
!unzip ./data.zip
!rm ./data.zip

     |████████████████████████████████| 532kB 7.4MB/s 
Downloading...
From: https://drive.google.com/uc?id=1fHi1ko7wr80wXkXpqpqpOxuYH1mClXoX
To: /content/data.zip
100% 490k/490k [00:00<00:00, 66.8MB/s]
Archive:  ./data.zip
   creating: data/
   creating: data/deer/
 extracting: data/deer/deer13.png    
 extracting: data/deer/deer6.png     
 extracting: data/deer/deer11.png    
 extracting: data/deer/deer2.png     
 extracting: data/deer/deer10.png    
 extracting: data/deer/deer16.png    
 extracting: data/deer/deer9.png     
 extracting: data/deer/deer20.png    
 extracting: data/deer/deer15.png    
 extracting: data/deer/deer19.png    
 extracting: data/deer/deer5.png     
 extracting: data/deer/deer14.png    
 extracting: data/deer/deer4.png     
 extracting: data/deer/deer8.png     
 extracting: data/deer/deer12.png    
 extracting: data/deer/deer1.png     
 extracting: data/deer/deer7.png     
 extracting: data/deer/deer17.png    
 extracting: data/deer/deer18.png    
 extracting:

## Global Settings

* $\epsilon$ is fixed to be 8. But on **Data section**, we will first apply transforms on raw pixel value (0-255 scale) **by ToTensor (to 0-1 scale)** and then **Normalize (subtract mean divide std)**. $\epsilon$ should be set to $\frac{8}{255 * std}$ during attack.

* Explaination (optional)
    * Denote the first pixel of original image as $p$, and the first pixel of adversarial image as $a$.
    * The $\epsilon$ constraints tell us $\left| p-a \right| <= 8$.
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.
    * After applying ToTensor() and Normalize() on $p$ and $a$, the constraint becomes $\left| N(T(p))-N(T(a)) \right| = \left| \frac{\frac{p}{255}-mean}{std}-\frac{\frac{a}{255}-mean}{std} \right| = \frac{1}{255 * std} \left| p-a \right| <= \frac{8}{255 * std}.$
    * So, we should set $\epsilon$ to be $\frac{8}{255 * std}$ after ToTensor() and Normalize().

In [ ]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size = 8

# the mean and std are the calculated statistics from cifar_10 dataset
cifar_10_mean = (0.491, 0.482, 0.447) # mean for the three channels of cifar_10 images
cifar_10_std = (0.202, 0.199, 0.201) # std for the three channels of cifar_10 images

# convert mean and std to 3-dimensional tensors for future operations
mean = torch.tensor(cifar_10_mean).to(device).view(3, 1, 1)
std = torch.tensor(cifar_10_std).to(device).view(3, 1, 1)

epsilon = 8/255/std
# TODO: iterative fgsm attack
# alpha (step size) can be decided by yourself
# alpha = 0.8/255/std
num_iter = 50
alpha = epsilon/10

root = './data' # directory for storing benign images
# benign images: images which do not contain adversarial perturbations
# adversarial images: images which include adversarial perturbations

## Data

Construct dataset and dataloader from root directory. Note that we store the filename of each image for future usage.

In [ ]:
import os
import glob
import shutil
import numpy as np
from PIL import Image
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_10_mean, cifar_10_std)
])

class AdvDataset(Dataset):
    def __init__(self, data_dir, transform):
        self.images = []
        self.labels = []
        self.names = []
        '''
        data_dir
        ├── class_dir
        │   ├── class1.png
        │   ├── ...
        │   ├── class20.png
        '''
        for i, class_dir in enumerate(sorted(glob.glob(f'{data_dir}/*'))):
            images = sorted(glob.glob(f'{class_dir}/*'))
            self.images += images
            self.labels += ([i] * len(images))
            self.names += [os.path.relpath(imgs, data_dir) for imgs in images]
        self.transform = transform
    def __getitem__(self, idx):
        image = self.transform(Image.open(self.images[idx]))
        label = self.labels[idx]
        return image, label
    def __getname__(self):
        return self.names
    def __len__(self):
        return len(self.images)

adv_set = AdvDataset(root, transform=transform)
adv_names = adv_set.__getname__()
adv_loader = DataLoader(adv_set, batch_size=batch_size, shuffle=False)

print(f'number of images = {adv_set.__len__()}')

number of images = 200


## Utils -- Benign Images Evaluation

In [ ]:
# to evaluate the performance of model on benign images
def epoch_benign(model, loader, loss_fn):
    model.eval()
    train_acc, train_loss = 0.0, 0.0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        yp = model(x)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
    return train_acc / len(loader.dataset), train_loss / len(loader.dataset)

## Utils -- Attack Algorithm

In [ ]:
# perform fgsm attack
def fgsm(model, x, y, loss_fn, epsilon=epsilon):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    x_adv.requires_grad = True # need to obtain gradient of x_adv, thus set required grad
    loss = loss_fn(model(x_adv), y) # calculate loss
    loss.backward() # calculate gradient
    # fgsm: use gradient ascent on x_adv to maximize loss
    x_adv = x_adv + epsilon * x_adv.grad.detach().sign() 
    return x_adv

# TODO: perform iterative fgsm attack
# set alpha as the step size in Global Settings section
# alpha and num_iter can be decided by yourself
def ifgsm(model, x, y, loss_fn, epsilon=epsilon, alpha=alpha, num_iter=num_iter):
    x_adv = x.detach().clone() # initialize x_adv as original benign image x
    # write a loop of num_iter to represent the iterative times
    for i in range(num_iter):
      # call fgsm with (epsilon = alpha) to obtain new x_adv
      x_adv = fgsm(model, x_adv, y, loss_fn, epsilon=alpha)
      # clip new x_adv back to [x-epsilon, x+epsilon]
      x_adv = torch.max(torch.min(x_adv,x+epsilon),x-epsilon)

    return x_adv  
    # pass


# R+FGSM Attack ref:(https://github.com/Harry24k/RFGSM-pytorch/blob/master/RFGSM.ipynb)
def rfgsm(model, images, labels, loss_fn, epsilon=epsilon, alpha=alpha) :    

    outputs = model(images)
    _, labels = torch.min(outputs.data, 1)
    labels = labels.detach_()
    
    images_new = images + alpha*torch.randn_like(images).sign()
    images_new.requires_grad = True
    
    outputs = model(images_new)

    model.zero_grad()
    cost = loss_fn(outputs, labels).to(device)
    cost.backward()
    
    attack_images = images_new + (epsilon-alpha)*images_new.grad.sign()
    attack_images = torch.clamp(attack_images, 0, 1)

    return attack_images

## Utils -- Attack

* Recall
    * ToTensor() can be seen as a function where $T(x) = x/255$.
    * Normalize() can be seen as a function where $N(x) = (x-mean)/std$ where $mean$ and $std$ are constants.

* Inverse function
    * Inverse Normalize() can be seen as a function where $N^{-1}(x) = x*std+mean$ where $mean$ and $std$ are constants.
    * Inverse ToTensor() can be seen as a function where $T^{-1}(x) = x*255$.

* Special Noted
    * ToTensor() will also convert the image from shape (height, width, channel) to shape (channel, height, width), so we also need to transpose the shape back to original shape.
    * Since our dataloader samples a batch of data, what we need here is to transpose **(batch_size, channel, height, width)** back to **(batch_size, height, width, channel)** using np.transpose.

In [ ]:
# perform adversarial attack and generate adversarial examples
def gen_adv_examples(model, loader, attack, loss_fn):
    model.eval()
    adv_names = []
    train_acc, train_loss = 0.0, 0.0
    for i, (x, y) in enumerate(loader):
        x, y = x.to(device), y.to(device)
        x_adv = attack(model, x, y, loss_fn) # obtain adversarial examples
        yp = model(x_adv)
        loss = loss_fn(yp, y)
        train_acc += (yp.argmax(dim=1) == y).sum().item()
        train_loss += loss.item() * x.shape[0]
        # store adversarial examples
        adv_ex = ((x_adv) * std + mean).clamp(0, 1) # to 0-1 scale
        adv_ex = (adv_ex * 255).clamp(0, 255) # 0-255 scale
        adv_ex = adv_ex.detach().cpu().data.numpy().round() # round to remove decimal part
        adv_ex = adv_ex.transpose((0, 2, 3, 1)) # transpose (bs, C, H, W) back to (bs, H, W, C)
        adv_examples = adv_ex if i == 0 else np.r_[adv_examples, adv_ex]
    return adv_examples, train_acc / len(loader.dataset), train_loss / len(loader.dataset)

# create directory which stores adversarial examples
def create_dir(data_dir, adv_dir, adv_examples, adv_names):
    if os.path.exists(adv_dir) is not True:
        _ = shutil.copytree(data_dir, adv_dir)
    for example, name in zip(adv_examples, adv_names):
        im = Image.fromarray(example.astype(np.uint8)) # image pixel value should be unsigned int
        im.save(os.path.join(adv_dir, name))

## Model / Loss Function

Model list is available [here](https://github.com/osmr/imgclsmob/blob/master/pytorch/pytorchcv/model_provider.py). Please select models which has _cifar10 suffix. Some of the models cannot be accessed/loaded. You can safely skip them since TA's model will not use those kinds of models.

In [ ]:
# """ref:https://www.kaggle.com/iamsdt/ensemble-model-pytorch"""
# import torch.nn.functional as F
# class MyEnsemble(nn.Module):

#     def __init__(self, models, input):
#         super(MyEnsemble, self).__init__()
#         self.models = models
#         # cifar10
#         self.fc1 = nn.Linear(input, 10)

#     def forward(self, x):
#         out = [model(x.clone()) for model in self.models]
#         out = tuple(out)
#         x = torch.cat(out, dim=1)
#         x = self.fc1(x)

#         return torch.softmax(x)

In [ ]:
from pytorchcv.model_provider import get_model as ptcv_get_model
# models_name = ['resnet110_cifar10','wrn20_10_32bit_cifar10','nin_cifar10','rir_cifar10',\
#                'seresnet56_cifar10','ror3_110_cifar10','pyramidnet110_a48_cifar10']
# models = [ptcv_get_model(name, pretrained=True).to(device) for name in models_name]
# model = MyEnsemble(models, len(models_name)*10)


# model = ptcv_get_model('resnet20_cifar10', pretrained=True).to(device)
# model = ptcv_get_model('resnet56_cifar10', pretrained=True).to(device)
# model = ptcv_get_model('resnet110_cifar10', pretrained=True).to(device)
# model = ptcv_get_model('resnet272bn_cifar10', pretrained=True).to(device) # 0.41
# model = ptcv_get_model('resnet542bn_cifar10', pretrained=True).to(device) # 0.48
# model = ptcv_get_model('diaresnet56_cifar10', pretrained=True).to(device) # 0.37
# model = ptcv_get_model('diapreresnet110_cifar10', pretrained=True).to(device) # 0.25
# model = ptcv_get_model('diapreresnet56_cifar10', pretrained=True).to(device) # 0.37
# model = ptcv_get_model('wrn20_10_32bit_cifar10', pretrained=True).to(device) # 
# model = ptcv_get_model('nin_cifar10', pretrained=True).to(device) #0.280
# model = ptcv_get_model('rir_cifar10', pretrained=True).to(device)
# model = ptcv_get_model('ror3_110_cifar10', pretrained=True).to(device) # 0.44
# model = ptcv_get_model('seresnet20_cifar1 0', pretrained=True).to(device) # 0.56
# model = ptcv_get_model('seresnet56_cifar10', pretrained=True).to(device) # 0.56
# model = ptcv_get_model('seresnet110_cifar10', pretrained=True).to(device) # 0.56
# model = ptcv_get_model('seresnet164bn_cifar10', pretrained=True).to(device) # 0.56
# model = ptcv_get_model('seresnet272bn_cifar10', pretrained=True).to(device) # 0.56

# model = ptcv_get_model('seresnet542bn_cifar10', pretrained=True).to(device) # 0.56

# model = ptcv_get_model('sepreresnet20_cifar10', pretrained=True).to(device) # 0.43
# model = ptcv_get_model('sepreresnet56_cifar10', pretrained=True).to(device) # 0.43
# model = ptcv_get_model('sepreresnet110_cifar10', pretrained=True).to(device) # 0.23
# model = ptcv_get_model('sepreresnet542bn_cifar10', pretrained=True).to(device) # 0.45
# model = ptcv_get_model('sepreresnet272bn_cifar10', pretrained=True).to(device) # 0.45
# model = ptcv_get_model('sepreresnet1202_cifar10', pretrained=True).to(device) # 0.45

# model = ptcv_get_model('pyramidnet110_a48_cifar10', pretrained=True).to(device) # 0.45
# model = ptcv_get_model('pyramidnet110_a84_cifar10', pretrained=True).to(device) # 0.45
# model = ptcv_get_model('pyramidnet110_a270_cifar10', pretrained=True).to(device) # 0.45
  


model = ptcv_get_model('preresnet20_cifar10', pretrained=True).to(device) # 0.25
# model = ptcv_get_model('preresnet56_cifar10', pretrained=True).to(device) # 0.25
# model = ptcv_get_model('preresnet110_cifar10', pretrained=True).to(device) # 0.25
# model = ptcv_get_model('xdensenet40_2_k36_bc_cifar10', pretrained=True).to(device) # 0.44
# model = ptcv_get_model('xdensenet40_2_k24_bc_cifar10', pretrained=True).to(device) # 0.46
# model = ptcv_get_model('shakeshakeresnet26_2x32d_cifar10', pretrained=True).to(device) # 0.26
# model = ptcv_get_model('densenet190_k40_bc_cifar10', pretrained=True).to(device) # 0.54
# model = ptcv_get_model('densenet40_k36_bc_cifar10', pretrained=True).to(device) # 
# model = ptcv_get_model('densenet40_k12_bc_cifar10', pretrained=True).to(device) # 0.6
# model = ptcv_get_model('diapreresnet20_cifar10', pretrained=True).to(device) # 0.52
# model = ptcv_get_model('preresnet164bn_cifar10', pretrained=True).to(device) # 0.51
# model = ptcv_get_model('preresnet272bn_cifar10', pretrained=True).to(device) # 0.41
# model = ptcv_get_model('preresnet1001_cifar10', pretrained=True).to(device) # 0.
# model = ptcv_get_model('preresnet1001_cifar10', pretrained=True).to(device) # 0.57
# model = ptcv_get_model('resnext29_32x4d_cifar10', pretrained=True).to(device) # 0.57
# model = ptcv_get_model('resnext29_16x64d_cifar10', pretrained=True).to(device) # 0.62
# model = ptcv_get_model('resnext272_1x64d_cifar10', pretrained=True).to(device) # 0.57
# model = ptcv_get_model('resnext272_2x32d_cifar10', pretrained=True).to(device) # 0.57

In [ ]:
# model.to(device)
loss_fn = nn.CrossEntropyLoss()

benign_acc, benign_loss = epoch_benign(model, adv_loader, loss_fn)
print(f'benign_acc = {benign_acc:.5f}, benign_loss = {benign_loss:.5f}')

benign_acc = 0.97000, benign_loss = 0.14476


## FGSM

In [ ]:
# adv_examples, fgsm_acc, fgsm_loss = gen_adv_examples(model, adv_loader, fgsm, loss_fn)
# print(f'fgsm_acc = {fgsm_acc:.5f}, fgsm_loss = {fgsm_loss:.5f}')

# create_dir(root, 'fgsm', adv_examples, adv_names)

## I-FGSM

In [ ]:
# TODO: iterative fgsm attack
adv_examples, ifgsm_acc, ifgsm_loss = gen_adv_examples(model, adv_loader, ifgsm, loss_fn)

print(f'ifgsm_acc = {ifgsm_acc:.10f}, ifgsm_loss = {ifgsm_loss:.5f}')
print('best_ifgsm_loss =  17.03254')
create_dir(root, 'ifgsm', adv_examples, adv_names)

# rfgsm attack
# adv_examples, rfgsm_acc, rfgsm_loss = gen_adv_examples(model, adv_loader, rfgsm, loss_fn)

# print(f'rfgsm_acc = {rfgsm_acc:.10f}, rfgsm_loss = {rfgsm_loss:.5f}')
# print('best_rfgsm_loss =  17.03254')
# create_dir(root, 'rfgsm', adv_examples, adv_names)

ifgsm_acc = 0.0100000000, ifgsm_loss = 48.85788
best_ifgsm_loss =  17.03254


## Compress the images

In [ ]:
# %cd fgsm
# !tar zcvf ../fgsm.tgz *
# %cd ..

%cd ifgsm
!tar zcvf ../ifgsm.tgz *
%cd ..

# %cd rfgsm
# !tar zcvf ../rfgsm.tgz *
# %cd ..

/content/ifgsm
airplane/
airplane/airplane12.png
airplane/airplane20.png
airplane/airplane17.png
airplane/airplane4.png
airplane/airplane2.png
airplane/airplane1.png
airplane/airplane7.png
airplane/airplane11.png
airplane/airplane5.png
airplane/airplane3.png
airplane/airplane10.png
airplane/airplane19.png
airplane/airplane9.png
airplane/airplane8.png
airplane/airplane13.png
airplane/airplane16.png
airplane/airplane15.png
airplane/airplane14.png
airplane/airplane18.png
airplane/airplane6.png
automobile/
automobile/automobile20.png
automobile/automobile5.png
automobile/automobile2.png
automobile/automobile7.png
automobile/automobile16.png
automobile/automobile3.png
automobile/automobile1.png
automobile/automobile8.png
automobile/automobile17.png
automobile/automobile11.png
automobile/automobile15.png
automobile/automobile4.png
automobile/automobile14.png
automobile/automobile19.png
automobile/automobile9.png
automobile/automobile18.png
automobile/automobile12.png
automobile/automobile10.

## Visualization

In [ ]:
# import matplotlib.pyplot as plt

# classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# error = 0
# for j in range(1,21):
#   plt.figure(figsize=(20, 20))
#   cnt = 0
#   for i, cls_name in enumerate(classes):
#       path = f'{cls_name}/{cls_name}{j}.png'
#       # benign image
#       cnt += 1
#       plt.subplot(len(classes), 10, cnt)
#       im = Image.open(f'./data/{path}')
#       logit = model(transform(im).unsqueeze(0).to(device))[0]
#       predict = logit.argmax(-1).item()
#       prob = logit.softmax(-1)[predict].item()
#       plt.title(f'benign: {cls_name}{j}\n{classes[predict]}: {prob:.4%}')
#       plt.axis('off')
#       plt.imshow(np.array(im))
#       temp1 = classes[predict]
#       # adversarial image
#       cnt += 1
#       plt.subplot(len(classes), 10, cnt)
#       im = Image.open(f'./ifgsm/{path}')
#       logit = model(transform(im).unsqueeze(0).to(device))[0]
#       predict = logit.argmax(-1).item()
#       prob = logit.softmax(-1)[predict].item()
#       plt.title(f'adversarial: {cls_name}{j}\n{classes[predict]}: {prob:.4%}')
#       temp2 = classes[predict]
#       if temp1==temp2:
#         print('!!!!!!',temp1,temp2,flush=True)
#         print(f'{cls_name}{j}')
#         error+=1
#       plt.axis('off')
#       plt.imshow(np.array(im))
#   plt.tight_layout()
#   plt.show()
#   print('--')

In [ ]:
print(str(ifgsm_acc))
# print(error)



0.01
